
<center><h1>Medallion Architecture con PySpark</h1>

En este notebook se trabajará con una serie de archivos .csv, siguiendo la **Medallion Architecture**.

En la **capa bronce** se alojarán los archivos .csv tal cual fueron extraidos del origen.

En la **capa silver** se alojarán los archivos en formato parquet, con las modificaciones pertinentes.
</center>


<p align="center">
 <img src=https://media.licdn.com/dms/image/C4E12AQFDZNNpFYcwLQ/article-inline_image-shrink_1000_1488/0/1628283147919?e=1689206400&v=beta&t=WlxNVNtfJ6qQqIZqT-8PTJxZjQf5bvZkTzTQRb9kKnE width=200px>
 </p>

# **0. Instalación de Apache Spark en Google Colab**

<p align="center">
  <img src="https://www.vectorlogo.zone/logos/apache_spark/apache_spark-ar21.png" width="200px">
</p>

## 0.1 Instalamos y actualizamos PySpark

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=b4bdda1b13d294ff06dddc33d4e8a46785611ab40e624102e44351acf8a3e032
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
!pip install --upgrade pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 0.2 Importamos/creamos la SparkSession

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('myAppName').getOrCreate()

## 0.3 Importamos los datatypes necesarios para armar el schema

In [4]:
from pyspark.sql.types import IntegerType, StringType, DoubleType, StructField, StructType

# **1. Trabajamos con 'orders.csv' de la capa bronce**

<p align="center">
<img src=https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSaAFeb45X1cH0uyReSZXaPxvs-jvoisalsCw&usqp=CAU width=200px>

## 1.1 Definimos el path del csv e importamos el schema

In [5]:
orders_path = '/content/bronze/orders.csv'

orders_schema = StructType([
                    StructField('ORDER_ID', IntegerType(), False),
                    StructField('ORDER_DATETIME', StringType(), False),
                    StructField('CUSTOMER_ID', IntegerType(), False),
                    StructField('ORDER_STATUS', StringType(), False),
                    StructField('STORE_ID', IntegerType(), False)
                    ]
                    )

orders_df = spark.read.csv(path=orders_path, header=True, schema=orders_schema)

## 1.2 Importamos la función 'to_timestamp' que va a ser necesario para castear la columna 'ORDER_DATETIME' y hacemos el casteo en 'ORDER_TIMESTAMP'

In [6]:
from pyspark.sql.functions import to_timestamp

In [7]:
orders_df = orders_df.select('ORDER_ID', \
                             to_timestamp(orders_df['ORDER_DATETIME'], 'dd-MMM-yy kk.mm.ss.SS').alias('ORDER_TIMESTAMP'), \
                             'CUSTOMER_ID', \
                             'ORDER_STATUS', \
                             'STORE_ID'
                            )

## 1.3 Filtramos 'ORDER_STATUS' para que solo queden los que están completados

In [8]:
orders_df = orders_df.filter(orders_df['ORDER_STATUS']=='COMPLETE')

## 1.4 Importar 'stores.csv' y hacer un JOIN, para mostrar los nombres de las tiendas, en lugar de los ID

In [9]:
stores_path = '/content/bronze/stores.csv'

stores_schema = StructType([
                            StructField('STORE_ID', IntegerType(), False),
                            StructField('STORE_NAME', StringType(), False),
                            StructField('WEB_ADDRESS', StringType(), False),
                            StructField('LATITUDE', DoubleType(), False),
                            StructField('LONGITUDE', DoubleType(), False),
                            ]
                            )

stores_df = spark.read.csv(path=stores_path, header=True, schema=stores_schema)

## 1.5 Hacemos un LEFT JOIN de orders con stores para agregar 'store_name' al dataframe. Seleccionamos sólo las columnas necesarias

In [10]:
orders_df = orders_df.join(stores_df, orders_df['store_id']==stores_df['store_id'], 'left').select('ORDER_ID','ORDER_TIMESTAMP','CUSTOMER_ID','STORE_NAME')

## 1.6 Sobreescribimos los archivos en la capa 'Silver' como un archivo Parquet

In [11]:
orders_df.write.parquet('/content/silver/orders', mode='overwrite')

# **2. Trabajamos con 'order_items.csv' de la capa bronce**



<p align="center">
<img src=https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSaAFeb45X1cH0uyReSZXaPxvs-jvoisalsCw&usqp=CAU width=200px>

## 2.1 Definimos el path e importamos el schema

In [12]:
order_items_path = '/content/bronze/order_items.csv'

order_items_schema = StructType([
                                  StructField('ORDER_ID', IntegerType(), False),
                                  StructField('LINE_ITEM_ID', StringType(), False),
                                  StructField('PRODUCT_ID', IntegerType(), False),
                                  StructField('UNIT_PRICE', DoubleType(), False),
                                  StructField('QUANTITY', IntegerType(), False)
                                ]
                                )

order_items_df = spark.read.csv(path=order_items_path, header=True, schema=order_items_schema)

## 2.2 Seleccionamos las columnas necesarias y dropeamos las que no vamos a usar

In [13]:
order_items_df = order_items_df.drop('LINE_ITEM_ID')

## 2.3 Sobreescribimos 'order_items' en la capa Silver como un archivo Parquet

In [14]:
order_items_df.write.parquet('/content/silver/order_items', mode='overwrite')

# **3. Trabajamos con 'products.csv' de la capa bronce**

<p align="center">
<img src=https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSaAFeb45X1cH0uyReSZXaPxvs-jvoisalsCw&usqp=CAU width=200px>

## 3.1 Definimos el path e importamos el schema

In [15]:
products_path = '/content/bronze/products.csv'

products_schema = StructType([
                              StructField('PRODUCT_ID', IntegerType(), False),
                              StructField('PRODUCT_NAME', StringType(), False),
                              StructField('UNIT_PRICE', DoubleType(), False)
                             ]
                             )

products_df = spark.read.csv(path=products_path, header=True, schema=products_schema)

## 3.2 Sobreescribimos 'products' en la capa Silver como un archivo Parquet

In [16]:
products_df.write.parquet('/content/silver/products', mode='overwrite')

# **4. Trabajamos con 'customers.csv' de la capa bronce**

<p align="center">
<img src=https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSaAFeb45X1cH0uyReSZXaPxvs-jvoisalsCw&usqp=CAU width=200px>

## 4.1 Definimos el path e importamos el schema

In [17]:
customers_path = '/content/bronze/customers.csv'

customers_schema = StructType([
                                StructField('CUSTOMER_ID', IntegerType(), False),
                                StructField('FULL_NAME', StringType(), False),
                                StructField('EMAIL_ADRESS', StringType(), False)
                              ]
                              )

customers_df = spark.read.csv(path=customers_path, header=True, schema=customers_schema)

## 4.2 Sobreescribimos 'customers' en la capa Silver como un archivo Parquet

In [18]:
customers_df.write.parquet('/content/silver/customers', mode='overwrite')

<center><h1><b>Fase II: De Silver a Gold</b></h1></center>

<center>En esta etapa se realizará el enriquecimiento de los datos previamente tratados. Se importarán los archivos en formato parquet desde la carpeta 'silver', previamente creada.

## 0.1 Definimos el path y leemos los Parquet desde la capa Silver

In [19]:
customers_silver_path = '/content/silver/customers'
order_items_silver_path = '/content/silver/order_items'
orders_silver_path = '/content/silver/orders'
products_silver_path = '/content/silver/products'

customers_silver_df = spark.read.parquet(customers_silver_path)
order_items_silver_df = spark.read.parquet(order_items_silver_path)
orders_silver_df = spark.read.parquet(orders_silver_path)
products_silver_df = spark.read.parquet(products_silver_path)